In [1]:
from openai import OpenAI
import os

from dotenv import load_dotenv

In [2]:
#loade the openai key from .env

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [8]:
from langchain.document_loaders import DirectoryLoader
DATA_PATH = 'data/'

def load_documents():
    loader = DirectoryLoader(DATA_PATH, glob= "*.md")
    documents = loader.load()
    return documents



In [9]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size= 1000,
    cunk_overlap= 500,
    length_function=len,
    add_start_index=True,
)

chunks = text_splitter.split_documents(documents)

NameError: name 'RecursiveCharacterTextSplitter' is not defined